In [ ]:
# 在本机未更新至2.0.8的情况下，使用vnpy2.0.8的回测逻辑
import sys
from pathlib import Path
new_version_path = Path(r'D:\vnpy-2.0.8')
# new_version_path = Path(r'E:\vnpy\vnpy-2.0.8')
sys.path.insert(0, str(new_version_path))
# sys.path

import vnpy
print(vnpy.__version__)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
plt.style.use('ggplot')

from datetime import datetime, timedelta
from typing import Tuple
from vnpy.app.cta_strategy.backtesting import BacktestingEngine, OptimizationSetting
from double_ma_strategy import DoubleMaStrategy
from utility import (vt_bar_to_df, vt_trade_to_df, load_data, strip_digt,
                     get_dominant_in_periods, clear_open_trade_after_deadline,
                     single_backtest, process_last_trade_dt)


In [ ]:
# 从数据库载入数据
price = load_data('RB888.SHFE', '1h', datetime(2019, 1, 1), datetime(2019, 12, 1))
print(f"记录数：{len(price)} 重复数：{sum(price.index.duplicated())}")
price.head()

In [ ]:
# 用matplot的交互式窗口查看数据是否正常
# 按住右键上下左右移动可以上下左右缩放，按住左键可以平移
price2 = price.reset_index()
price2.head()

%matplotlib notebook
axe = price2.close.plot(figsize=(12, 6))
axe.scatter([2, 10], [2596, 2612], color='b')
plt.show()

In [ ]:
# 设置回测参数

engine = BacktestingEngine()
engine.set_parameters(
    vt_symbol="RB888.SHFE",
    interval="1h",
    start=datetime(2019, 1, 1),
    end=datetime(2019, 12, 1),
    rate=1/10000,
    slippage=1,
    size=10,
    pricetick=1,
    capital=100000,
)
engine.add_strategy(DoubleMaStrategy, {})

In [ ]:
# 运行回测
engine.load_data()
engine.run_backtesting()
df = engine.calculate_result()
df.to_csv('result.csv')
# engine.calculate_statistics()
# engine.show_chart()

In [ ]:
# 获取成交数据并在收盘价上绘制成交点
trades = engine.get_all_trades()
trade_df = vt_trade_to_df(trades)
long_df = trade_df[trade_df.direction == '多']
short_df = trade_df[trade_df.direction == '空']

# 计算成交点x,y序列
price['id'] = range(len(price))
long_x = price.loc[long_df.index].id.values
long_y = long_df.price.values
short_x = price.loc[short_df.index].id.values
short_y = short_df.price.values


In [ ]:
trade_df

In [ ]:
trade_list = clear_open_trade_after_deadline(trades, deadline=datetime(2019, 8, 26))
trade_list

In [ ]:
# 绘制数据
%matplotlib notebook
price2 = price.reset_index()
axe = price2.close.plot(figsize=(12, 6), color='#a8a8a8', zorder=10)
axe.scatter(long_x, long_y, color='r', marker=6, zorder=20)
axe.scatter(short_x, short_y, color='g', marker=7, zorder=30)
plt.show()

In [ ]:
# 进行连续合约的回测

### 进行分段合约的回测

- 这种方式适用于某个合约在【变成非主力-到期】这个区间，策略一定会发出平仓信号，可以自然切换到下一个主力合约，基本上就是那些持仓周期较短的策略。对于合约快到期了，但是策略还在持仓中，需要倒仓到下个主力合约的策略，倒仓的逻辑会复杂一些，暂时不适用这种方式。

- 对于永远在场的策略也不适合，比如双均线策略，原因同上，因为快到期了，策略仍然持仓，需要人工倒仓。

In [ ]:
# 载入分段主力合约数据
backtest_start = datetime(2018, 2, 10)
backtest_end = datetime(2019, 12, 10)

# 载入主力合约起止日期数据
seg = pd.read_csv('dominant_data.csv', parse_dates=[1, 2])
# seg['underlying'] = seg['dominant'].map(lambda x: strip_digt(x))

# 找出主力合约的起点
sel = seg[seg['underlying'] == 'RB'].copy()
# sel['vt_symbol'] = sel['dominant'].map(lambda x: x + '.')
# sel['vt_symbol'] = sel['vt_symbol'] + sel['exchange']
# sel['vt_symbol'] = sel['vt_symbol'].map(to_vt_symbol)
sel.reset_index(inplace=True)
passed = sel[sel['start'] < backtest_start]
after = sel[sel['start'] > backtest_end]

passed_closest_idx = passed.index.values[-1]
after_first_idx = after.index.values[0] if not after.empty else len(sel)

# 选出的合约如果变非主力的日期和回测开始日期相比，只剩几天（不够初始化历史数据）应该排除。
# 过滤天数：30天计算指标的历史数据 + 最少可交易7天（相当于1周交易天数）
if passed.iloc[-1]['end'] - backtest_start < timedelta(days=37):
    passed_closest_idx += 1
sel[passed_closest_idx: after_first_idx]


In [ ]:
backtest_start = datetime(2018, 1, 1)
backtest_end = datetime(2019, 10, 1)
commodity = 'RB'

dom_df = get_dominant_in_periods(commodity, backtest_start, backtest_end)
dom_df

In [ ]:
start = backtest_start
btest_dfs = []
for (idx, row) in dom_df.iterrows():
    # even become sub-main, but if open trade exists, it must continute until no position.
    end = row['end'] 
    vt_symbol = row['vt_symbol']
    if idx == len(dom_df) - 1:
        end = backtest_end
        
    # run backtest function
    # the open trade after sub-main day must be deleted.
    df, last_trade_dt = single_backtest(vt_symbol, start, end)
    btest_dfs.append(df)
    
    # last_trade_dt to the closest open dt of last trade
    # backward xx days so as to init history data
    start = process_last_trade_dt(last_trade_dt)
    
all_df = pd.concat(btest_dfs)
    

In [ ]:
to_vt_symbol('RB2005.SHFE')
to_vt_symbol('A2005.DCE')
to_vt_symbol('AP2005.CZCE')
to_vt_symbol('SC2005.INE')
to_vt_symbol('IF2005.CFFEX')

In [ ]:
import datetime

In [ ]:
datetime.time(10)